In [1]:
#changement de dossier
import os
os.chdir("d:/Bureau/PythonProject/packages/scientisttools/data/")

#importation des données
import pandas
df = pandas.read_excel("Olive_Oil_Candisc.xlsx",sheet_name="dataset")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CLASSE       569 non-null    object
 1   palmitic     569 non-null    int64 
 2   palmitoleic  569 non-null    int64 
 3   stearic      569 non-null    int64 
 4   oleic        569 non-null    int64 
 5   linoleic     569 non-null    int64 
 6   linolenic    569 non-null    int64 
 7   arachidic    569 non-null    int64 
 8   eicosenoic   569 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 40.1+ KB


In [2]:
display(df.head())

CLASSE  palmitic  palmitoleic  stearic  oleic  linoleic  linolenic  \
0  Centre_North      1400           90      270   7420       800          0   
1  Centre_North       990          120      250   7750       870         10   
2  Centre_North      1020          100      220   7530      1030          0   
3  Centre_North      1098           54      202   7945       595         42   
4  Centre_North      1120           80      260   7750       680         30   

   arachidic  eicosenoic  
0         20           2  
1         10           2  
2          0           3  
3         32           2  
4         80           3

In [3]:
#classes - distribution
df.CLASSE.value_counts()

South           322
Centre_North    150
Sardinia         97
Name: CLASSE, dtype: int64

In [4]:
#analyse factorielle discriminante
from scientisttools.discriminant_analysis import CANDISC

#instanciation
candisc = CANDISC(n_components=2,
                  target=['CLASSE'],
                  features_labels=list(df.columns[1:]),
                  row_labels=df.index,
                  priors=None,
                  parallelize=True)

#entraînement -- l'erreur apparaît ici...
candisc.fit(df)

CANDISC(features_labels=['palmitic', 'palmitoleic', 'stearic', 'oleic',
                         'linoleic', 'linolenic', 'arachidic', 'eicosenoic'],
        n_components=2, parallelize=True,
        row_labels=RangeIndex(start=0, stop=569, step=1), target=['CLASSE'])

In [5]:
candisc.likelihood_test_

statistic  DDL num.  DDL den.          Pr>F
0  320.983726      16.0    1118.0  1.110223e-16
1  184.272085       7.0     560.0  1.110223e-16

In [6]:
print(candisc.manova_)

                    Multivariate linear model
                                                                 
-----------------------------------------------------------------
         CLASSE          Value   Num DF   Den DF  F Value  Pr > F
-----------------------------------------------------------------
          Wilks' lambda  0.0320 16.0000 1118.0000 320.9837 0.0000
         Pillai's trace  1.5917 16.0000 1120.0000 272.8899 0.0000
 Hotelling-Lawley trace 10.7752 16.0000  911.1473 375.9385 0.0000
    Roy's greatest root  8.4718  8.0000  560.0000 593.0293 0.0000



In [7]:

display(pandas.DataFrame(candisc.eig_.T))

0         1         2          3
0  8.471847  6.168446  78.62322   78.62322
1  2.303401       NaN  21.37678  100.00000

In [8]:
candisc.univariate_test_statistis_

Std. Dev.  R-squared  Rsq/(1-Rsq)  F-statistic  \
palmitic     168.715167   0.461491     0.856978   242.524854   
palmitoleic   52.558263   0.395095     0.653152   184.841942   
stearic       36.795195   0.001728     0.001731     0.489942   
oleic        405.966522   0.526521     1.112025   314.703134   
linoleic     242.802985   0.449629     0.816955   231.198312   
linolenic     12.985725   0.312278     0.454076   128.503464   
arachidic     22.033442   0.337110     0.508547   143.918675   
eicosenoic    14.092942   0.797929     3.948758  1117.498522   

             Prob (F-statistic)  
palmitic           8.465810e-77  
palmitoleic        1.650063e-62  
stearic            6.129213e-01  
oleic              1.288711e-92  
linoleic           4.032628e-74  
linolenic          9.724383e-47  
arachidic          2.936859e-51  
eicosenoic        2.867939e-197

In [9]:
pandas.DataFrame(candisc.coef_,index=candisc.features_labels_,columns=candisc.dim_index_)

LD1       LD2
palmitic     0.002760  0.008854
palmitoleic  0.013092  0.018422
stearic     -0.002794  0.004253
oleic        0.000626  0.006239
linoleic     0.001131 -0.001260
linolenic    0.041077  0.005822
arachidic   -0.017345 -0.034675
eicosenoic   0.163051  0.010059

In [10]:
candisc.gcenter_

LD1       LD2
CLASSE                          
Centre_North -3.322164  1.843764
Sardinia     -3.302678 -2.860687
South         2.542498  0.002863

In [11]:
pandas.DataFrame(candisc.score_coef_,index=candisc.features_labels_,columns=candisc.classes_)

Centre_North  Sardinia     South
palmitic         0.007155 -0.034444  0.007043
palmitoleic     -0.009528 -0.095936  0.033338
stearic          0.017124 -0.002940 -0.007091
oleic            0.009422 -0.019916  0.001611
linoleic        -0.006080 -0.000129  0.002871
linolenic       -0.125729 -0.152316  0.104454
arachidic       -0.006309  0.156480 -0.044199
eicosenoic      -0.523138 -0.567281  0.414587

In [12]:
candisc.gdisto_

Centre_North   Sardinia      South
Centre_North      0.000000  22.132240  37.783184
Sardinia         22.132240   0.000000  42.366007
South            37.783184  42.366007   0.000000

In [13]:
candisc.tcorr_

LD1       LD2
palmitic     0.717538 -0.037627
palmitoleic  0.659883 -0.089794
stearic     -0.007083  0.049135
oleic       -0.629206  0.497264
linoleic     0.264460 -0.745062
linolenic    0.574829 -0.154918
arachidic    0.278118 -0.619875
eicosenoic   0.944513  0.003556

In [14]:
candisc.bcorr_

LD1       LD2
palmitic     0.996819 -0.148078
palmitoleic  0.979502 -0.268340
stearic     -0.160870  0.995704
oleic       -0.666629  0.789483
linoleic     0.287779 -0.975225
linolenic    0.927928 -0.435710
arachidic    0.340070 -0.961566
eicosenoic   0.999983 -0.063032

In [15]:
candisc.wcorr_

LD1       LD2
palmitic     0.317710 -0.028211
palmitoleic  0.275680 -0.063522
stearic     -0.002304  0.027057
oleic       -0.297115  0.397609
linoleic     0.115828 -0.552566
linolenic    0.225224 -0.102781
arachidic    0.110992 -0.418893
eicosenoic   0.682714  0.004352

In [16]:
candisc.global_performance_

Stat        Value       p-value
0      Wilks' Lambda     0.031960           NaN
1  Bartlett -- C(16)  1936.843029  0.000000e+00
2  Rao -- F(16,1118)   320.983726  1.110223e-16

In [17]:
candisc.anova_

sum_sq     df       mean_sq            F  \
palmitic    C(CLASSE)  7.461387e+06    2.0  3.730694e+06   242.524854   
            Residual   8.706623e+06  566.0  1.538273e+04          NaN   
palmitoleic C(CLASSE)  6.199144e+05    2.0  3.099572e+05   184.841942   
            Residual   9.491124e+05  566.0  1.676877e+03          NaN   
stearic     C(CLASSE)  1.329040e+03    2.0  6.645198e+02     0.489942   
            Residual   7.676784e+05  566.0  1.356322e+03          NaN   
oleic       C(CLASSE)  4.928835e+07    2.0  2.464418e+07   314.703134   
            Residual   4.432305e+07  566.0  7.830928e+04          NaN   
linoleic    C(CLASSE)  1.505603e+07    2.0  7.528014e+06   231.198312   
            Residual   1.842944e+07  566.0  3.256085e+04          NaN   
linolenic   C(CLASSE)  2.991039e+04    2.0  1.495520e+04   128.503464   
            Residual   6.587092e+04  566.0  1.163797e+02          NaN   
arachidic   C(CLASSE)  9.295763e+04    2.0  4.647881e+04   143.918675   
            Residual   1.827908e+05  566.0  3.229519e+02          NaN   
eicosenoic  C(CLASSE)  9.001523e+04    2.0  4.500761e+04  1117.498522   
            Residual   2.279583e+04  566.0  4.027532e+01          NaN   

                              PR(>F)    eta_sq  omega_sq  
palmitic    C(CLASSE)   8.465810e-77  0.461491  0.459151  
            Residual             NaN       NaN       NaN  
palmitoleic C(CLASSE)   1.650063e-62  0.395095  0.392538  
            Residual             NaN       NaN       NaN  
stearic     C(CLASSE)   6.129213e-01  0.001728 -0.001796  
            Residual             NaN       NaN       NaN  
oleic       C(CLASSE)   1.288711e-92  0.526521  0.524409  
            Residual             NaN       NaN       NaN  
linoleic    C(CLASSE)   4.032628e-74  0.449629  0.447249  
            Residual             NaN       NaN       NaN  
linolenic   C(CLASSE)   9.724383e-47  0.312278  0.309472  
            Residual             NaN       NaN       NaN  
arachidic   C(CLASSE)   2.936859e-51  0.337110  0.334376  
            Residual             NaN       NaN       NaN  
eicosenoic  C(CLASSE)  2.867939e-197  0.797929  0.796931  
            Residual             NaN       NaN       NaN